# Interactuar con la base de datos propia
Armamos la siguiente clase para conectarnos a la base y realizar distintas operadciones
Notar que para realizar la conexion necesitamos proporcionar un filepath de nuestras credenciales.
Definimos funciones para:
- ejecutar una query
- insertar un df directamente.
- obtener la informacion general de una tabla.
- insertar un df a traves de un copy de un csv. (esto es mucho mas rapido y eficiente que el insert)

In [ ]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

# Load environment variables from .env
class PostgresDB:
    def __init__(self, dotenv_path):
        # Cargar variables de entorno
        load_dotenv(dotenv_path)
        self.user = os.getenv("user")
        self.password = os.getenv("password")
        self.host = os.getenv("host")
        self.port = os.getenv("port")
        self.dbname = os.getenv("dbname")
        try:
            self.conn = psycopg2.connect(
                user=self.user,
                password=self.password,
                host=self.host,
                port=self.port,
                dbname=self.dbname
            )
            self.cursor = self.conn.cursor()
            print("✅ Conexión exitosa a PostgreSQL")
        except Exception as e:
            print(f"❌ Error de conexión: {e}")

    def execute(self, query, params=None):
        """Ejecuta una query (INSERT, UPDATE, DELETE, DDL)."""
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            self.conn.commit()
        except Exception as e:
            print(f"❌ Error al ejecutar la query: {e}")

    def fetchall(self, query, params=None):
        """Ejecuta una consulta SELECT y devuelve todos los resultados."""
        try:
            if params:
                self.cursor.execute(query, params)
            else:
                self.cursor.execute(query)
            return self.cursor.fetchall()
        except Exception as e:
            print(f"❌ Error al ejecutar la consulta: {e}")
            return []

    def get_table_info(self, table_name):
        """Devuelve información de las columnas de una tabla."""
        try:
            self.cursor.execute(
                sql.SQL("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = %s;"),
                [table_name]
            )
            columns = self.cursor.fetchall()
            print(f"📋 Estructura de la tabla '{table_name}':")
            for col in columns:
                print(f"  - {col[0]} ({col[1]})")
            return columns
        except Exception as e:
            print(f"❌ Error al obtener información de la tabla '{table_name}': {e}")
            return []
        
    def insert_dataframe(self, df, table_name):
        """Inserta un DataFrame en la tabla especificada."""
        for index, row in df.iterrows():
            columns = ', '.join(df.columns)
            placeholders = ', '.join(['%s'] * len(df.columns))
            query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
            values = tuple(row.values)
            self.execute(query, values)
        print(f"✅ Insertados {len(df)} registros en la tabla '{table_name}'")

    def copy_from_csv(self, csv_path, table_name, columns=None):
        """Carga datos desde un CSV usando COPY (muy rápido para PostgreSQL)."""
        try:
            with open(csv_path, 'r') as file:
                self.cursor.copy_from(
                    file, 
                    table_name, 
                    sep=',', 
                    columns=columns  # Lista de nombres de columnas en el orden del CSV
                )
            self.conn.commit()
            print(f"✅ Datos cargados exitosamente desde {csv_path} a {table_name}")
            
        except Exception as e:
            print(f"❌ Error en COPY: {e}")
            self.conn.rollback()
        
        finally:
            # Limpiar archivo temporal
            import os
            if os.path.exists(csv_path):
                os.remove(csv_path)
                print(f"��️  Archivo temporal {csv_path} eliminado")

    def cerrar(self):
        """Cierra la conexión a la base de datos."""
        self.cursor.close()
        self.conn.close()
        print("Conexión cerrada.")


Ejemplo de uso: obtenemos los precios del año 2024 y realizamos un insert a traves de un copy.

Tambien vemos como hacer uso de la funcion creada en binane.py para obtener los precios

In [ ]:
from binance import obtener_precios_historicos
import pandas as pd

df = pd.DataFrame(obtener_precios_historicos("BTCUSDT", "1h", "2024-01-01", "2024-12-31"))

print(df.head())
# 2. Guardar a CSV
csv_path = "/Users/guidoboronat/personal/repo trading/personal_investments/personal_investments-1/Trading/Precios BTC/temp_btc_prices.csv"
df.to_csv(csv_path, index=False, header=True)

# creamos conexion y hacemos el copy
# dao = PostgresDB("/Users/guidoboronat/personal/repo trading/personal_investments/secrte.env")
# 3. Cargar con COPY
# columns = ['symbol', 'interval', 'open_time', 'open_price', 'high_price', 'low_price', 'close_price', 'volume', 'close_time', 'quote_volume', 'num_trades', 'taker_buy_base', 'taker_buy_quote']
# dao.copy_from_csv(csv_path, "btc_prices_1h", columns)
# dao.cerrar()

    symbol interval           open_time  open_price  high_price  low_price  \
0  BTCUSDT       1h 2024-01-01 00:00:00    42581.09    42586.64   42230.08   
1  BTCUSDT       1h 2024-01-01 01:00:00    42330.50    42399.99   42209.46   
2  BTCUSDT       1h 2024-01-01 02:00:00    42399.98    42406.00   42180.77   
3  BTCUSDT       1h 2024-01-01 03:00:00    42234.01    42424.82   42208.68   
4  BTCUSDT       1h 2024-01-01 04:00:00    42396.69    42500.00   42396.68   

   close_price     volume              close_time  quote_volume  num_trades  \
0     42330.49  794.80391 2024-01-01 00:59:59.999  3.370905e+07       38620   
1     42399.99  715.41760 2024-01-01 01:59:59.999  3.027162e+07       36038   
2     42234.01  736.53152 2024-01-01 02:59:59.999  3.115953e+07       32200   
3     42396.69  601.37250 2024-01-01 03:59:59.999  2.545412e+07       32653   
4     42492.46  653.67718 2024-01-01 04:59:59.999  2.775793e+07       27661   

   taker_buy_base  taker_buy_quote  
0       356.37209  

In [3]:
df.to_csv(csv_path, index=False, header=True)